Testing Postgres

In [27]:
import os
import uuid
import psycopg2
from psycopg2 import OperationalError, DatabaseError
from psycopg2.extras import DictCursor
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
from dotenv import load_dotenv
import random
import time

In [28]:
is_localhost = True

In [29]:
if is_localhost:
    load_dotenv('../.env')
else:
    load_dotenv()

In [30]:
os.environ['POSTGRES_DB']

'glimmerfox_db'

In [31]:
os.environ['RUN_TIMEZONE_CHECK'] = '0'

In [84]:
RUN_TIMEZONE_CHECK = os.getenv('RUN_TIMEZONE_CHECK', '1') == '1'

TZ_INFO = os.getenv("TZ", "Europe/Berlin")
tz = ZoneInfo(TZ_INFO)
tz

zoneinfo.ZoneInfo(key='Europe/Berlin')

In [33]:
def get_db_connection(localhost=False):

    if localhost:
        host=os.getenv("POSTGRES_HOST_LOCAL", "localhost")
    else:
        host=os.getenv("POSTGRES_HOST", "postgres")
    try:
        connection = psycopg2.connect(
            host=host,
            database=os.getenv("POSTGRES_DB", "glimmerfox_db"),
            user=os.getenv("POSTGRES_USER", "your_username"),
            password=os.getenv("POSTGRES_PASSWORD", "your_password"),
        )
        return connection
    except OperationalError as e:
        print(f"Error: Could not connect to the PostgreSQL database.\nDetails: {e}")
        return None


In [34]:
def init_db(localhost=False):
    conn = get_db_connection(localhost=localhost)
    if conn is None:
        print("Database connection failed.")
        return  # Exit the function if the connection failed

    try:
        with conn.cursor() as cur:
            cur.execute("DROP TABLE IF EXISTS feedback")
            cur.execute("DROP TABLE IF EXISTS queries")

            cur.execute("""
                CREATE TABLE queries (
                    id TEXT PRIMARY KEY,
                    question TEXT NOT NULL,
                    answer TEXT NOT NULL,
                    model_used TEXT NOT NULL,
                    response_time FLOAT NOT NULL,
                    relevance TEXT NOT NULL,
                    relevance_explanation TEXT NOT NULL,
                    input_tokens INTEGER NOT NULL,
                    output_tokens INTEGER NOT NULL,
                    total_tokens INTEGER NOT NULL,
                    eval_input_tokens INTEGER NOT NULL,
                    eval_output_tokens INTEGER NOT NULL,
                    eval_total_tokens INTEGER NOT NULL,
                    openai_cost FLOAT NOT NULL,
                    timestamp TIMESTAMP WITH TIME ZONE NOT NULL
                )
            """)
            print("Table 'queries' created successfully.")

            # Create 'feedback' table
            cur.execute("""
                CREATE TABLE feedback (
                    id SERIAL PRIMARY KEY,
                    query_id TEXT REFERENCES queries(id),
                    feedback INTEGER NOT NULL,
                    timestamp TIMESTAMP WITH TIME ZONE NOT NULL
                )
            """)
            print("Table 'feedback' created successfully.")

        # Commit the changes
        conn.commit()
        print("Database initialization completed successfully.")

        # Verify table creation by querying information_schema
        with conn.cursor() as cur:
            cur.execute("""
                SELECT table_name FROM information_schema.tables 
                WHERE table_schema = 'public'
            """)
            tables = cur.fetchall()
            print("Tables in the database:", tables)

    except DatabaseError as e:
        print(f"Database error: {e}")
        conn.rollback()  # Rollback in case of error

    finally:
        # Ensure the connection is always closed
        if conn:
            conn.close()
            print("Database connection closed.")

        conn.close()

In [35]:
def save_query(query_id, question, answer_data, timestamp=None, localhost=False):
    # change save_conversation -> save_query

    if timestamp is None:
        timestamp = datetime.now(tz)

    conn = get_db_connection(localhost=localhost)
    try:
        with conn.cursor() as cur:
            cur.execute(
                """
                INSERT INTO queries 
                (id, question, answer, model_used, response_time, relevance, 
                relevance_explanation, input_tokens, output_tokens, total_tokens, 
                eval_input_tokens, eval_output_tokens, eval_total_tokens, openai_cost, timestamp)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """,
                (
                    query_id,
                    question,
                    answer_data["answer"],
                    answer_data["model_used"],
                    answer_data["response_time"],
                    answer_data["relevance"],
                    answer_data["relevance_explanation"],
                    answer_data["input_tokens"],
                    answer_data["output_tokens"],
                    answer_data["total_tokens"],
                    answer_data["eval_input_tokens"],
                    answer_data["eval_output_tokens"],
                    answer_data["eval_total_tokens"],
                    answer_data["openai_cost"],
                    timestamp
                ),
            )
        conn.commit()
    finally:
        conn.close()


In [36]:
def save_feedback(query_id, feedback, timestamp=None, localhost=False):
    if timestamp is None:
        timestamp = datetime.now(tz)

    conn = get_db_connection(localhost=localhost)
    try:
        with conn.cursor() as cur:
            cur.execute(
                "INSERT INTO feedback (query_id, feedback, timestamp) VALUES (%s, %s, COALESCE(%s, CURRENT_TIMESTAMP))",
                (query_id, feedback, timestamp),
            )
        conn.commit()
    finally:
        conn.close()

In [37]:
def clear_tables(localhost=False):
    conn = get_db_connection(localhost=localhost)
    if conn is None:
        print("Database connection failed.")
        return

    try:
        with conn.cursor() as cur:
            cur.execute("DELETE FROM feedback")  # Clear the 'feedback' table first due to foreign key constraint
            cur.execute("DELETE FROM queries")   # Then clear the 'queries' table
        conn.commit()
        print("All entries in 'queries' and 'feedback' tables have been deleted.")
    except psycopg2.DatabaseError as e:
        print(f"Error deleting records from tables: {e}")
        conn.rollback()
    finally:
        if conn:
            conn.close()


In [38]:
def calculate_openai_cost(model_choice, tokens):
    openai_cost = 0

    if model_choice in ['openai/gpt-4o', 'openai/gpt-4o-mini']:
        openai_cost = (tokens['input_tokens'] * 0.000150 + tokens['output_tokens'] * 0.000600) / 1000

    return openai_cost

In [39]:
# # Function to generate sample queries
# def generate_sample_queries(n):
#     sample_queries = []
#     for i in range(n):
#         query_id = str(uuid.uuid4())
#         question = f"What is the meaning of life {i}?"
#         input_tokens = random.randint(234, 567)
#         output_tokens = random.randint(234, 567)
#         total_tokens = input_tokens + output_tokens
#         eval_input_tokens = random.randint(234, 567)
#         eval_output_tokens = random.randint(234, 567)
#         eval_total_tokens = eval_input_tokens + eval_output_tokens

#         answer_data = {
#             "answer": f"The meaning of life {i} is subjective.",
#             "model_used": "gpt-4o-mini",
#             "response_time": round(random.uniform(0.5, 2.5), 2),  # Random response time between 0.5 and 2.5 seconds
#             "relevance": random.choice(["NON_RELEVANT", "PARTLY_RELEVANT", "RELEVANT"]),
#             "relevance_explanation": f"The answer is {random.choice(['very', 'somewhat', 'not'])} relevant to the question.",
#             "input_tokens": input_tokens,  # Random token count between 10 and 50
#             "output_tokens": output_tokens,  # Random token count between 10 and 50
#             "total_tokens": total_tokens,  # Random total tokens
#             "eval_input_tokens": eval_input_tokens,  # Random token count for evaluation prompt
#             "eval_output_tokens": eval_output_tokens,  # Random token count for evaluation completion
#             "eval_total_tokens": eval_total_tokens,  # Random evaluation total tokens
#             # "openai_cost": round(random.uniform(0.01, 0.1), 4)  # Random cost between 0.01 and 0.1
#             "openai_cost": calculate_openai_cost(model_choice='openai/gpt-4o-mini', tokens={'input_tokens': input_tokens+eval_input_tokens, 'output_tokens': output_tokens+eval_output_tokens})
#         }
#         sample_queries.append((query_id, question, answer_data))
#     return sample_queries

In [80]:
# Function to generate sample queries
def generate_one_sample_query():
    query_id = str(uuid.uuid4())
    fake_number = datetime.now().timestamp()
    question = f"What is the meaning of life {fake_number}?"
    input_tokens = random.randint(234, 567)
    output_tokens = random.randint(234, 567)
    total_tokens = input_tokens + output_tokens
    eval_input_tokens = random.randint(234, 567)
    eval_output_tokens = random.randint(234, 567)
    eval_total_tokens = eval_input_tokens + eval_output_tokens

    answer_data = {
        "answer": f"The meaning of life {fake_number} is subjective.",
        "model_used": "gpt-4o-mini",
        "response_time": round(random.uniform(0.5, 2.5), 2),  # Random response time between 0.5 and 2.5 seconds
        # "relevance": random.choice(["NON_RELEVANT", "PARTLY_RELEVANT", "RELEVANT"]),
        "relevance": random.choices(["NON_RELEVANT", "PARTLY_RELEVANT", "RELEVANT"], weights=[1, 2, 4], k=1)[0],
        "relevance_explanation": f"The answer is {random.choice(['very', 'somewhat', 'not'])} relevant to the question.",
        "input_tokens": input_tokens,  # Random token count between 10 and 50
        "output_tokens": output_tokens,  # Random token count between 10 and 50
        "total_tokens": total_tokens,  # Random total tokens
        "eval_input_tokens": eval_input_tokens,  # Random token count for evaluation prompt
        "eval_output_tokens": eval_output_tokens,  # Random token count for evaluation completion
        "eval_total_tokens": eval_total_tokens,  # Random evaluation total tokens
        # "openai_cost": round(random.uniform(0.01, 0.1), 4)  # Random cost between 0.01 and 0.1
        "openai_cost": calculate_openai_cost(model_choice='openai/gpt-4o-mini', tokens={'input_tokens': input_tokens+eval_input_tokens, 'output_tokens': output_tokens+eval_output_tokens})
    }
    return (query_id, question, answer_data)

In [41]:
# def get_recent_conversations(limit=5, relevance=None):
#     conn = get_db_connection()
#     try:
#         with conn.cursor(cursor_factory=DictCursor) as cur:
#             query = """
#                 SELECT c.*, f.feedback
#                 FROM queries c
#                 LEFT JOIN feedback f ON c.id = f.query_id
#             """
#             if relevance:
#                 query += f" WHERE c.relevance = '{relevance}'"
#             query += " ORDER BY c.timestamp DESC LIMIT %s"

#             cur.execute(query, (limit,))
#             return cur.fetchall()
#     finally:
#         conn.close()

In [42]:
# def get_feedback_stats():
#     conn = get_db_connection()
#     try:
#         with conn.cursor(cursor_factory=DictCursor) as cur:
#             cur.execute("""
#                 SELECT 
#                     SUM(CASE WHEN feedback > 0 THEN 1 ELSE 0 END) as thumbs_up,
#                     SUM(CASE WHEN feedback < 0 THEN 1 ELSE 0 END) as thumbs_down
#                 FROM feedback
#             """)
#             return cur.fetchone()
#     finally:
#         conn.close()


In [43]:
# def check_timezone():
#     conn = get_db_connection()
#     try:
#         with conn.cursor() as cur:
#             cur.execute("SHOW timezone;")
#             db_timezone = cur.fetchone()[0]
#             print(f"Database timezone: {db_timezone}")

#             cur.execute("SELECT current_timestamp;")
#             db_time_utc = cur.fetchone()[0]
#             print(f"Database current time (UTC): {db_time_utc}")

#             db_time_local = db_time_utc.astimezone(tz)
#             print(f"Database current time ({TZ_INFO}): {db_time_local}")

#             py_time = datetime.now(tz)
#             print(f"Python current time: {py_time}")

#             # Use py_time instead of tz for insertion
#             cur.execute("""
#                 INSERT INTO queries 
#                 (id, question, answer, model_used, response_time, relevance, 
#                 relevance_explanation, input_tokens, output_tokens, total_tokens, 
#                 eval_input_tokens, eval_output_tokens, eval_total_tokens, openai_cost, timestamp)
#                 VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#                 RETURNING timestamp;
#             """, 
#             ('test', 'test question', 'test answer', 'test model', 0.0, 0.0, 
#              'test explanation', 0, 0, 0, 0, 0, 0, 0.0, py_time))

#             inserted_time = cur.fetchone()[0]
#             print(f"Inserted time (UTC): {inserted_time}")
#             print(f"Inserted time ({TZ_INFO}): {inserted_time.astimezone(tz)}")

#             cur.execute("SELECT timestamp FROM conversations WHERE id = 'test';")
#             selected_time = cur.fetchone()[0]
#             print(f"Selected time (UTC): {selected_time}")
#             print(f"Selected time ({TZ_INFO}): {selected_time.astimezone(tz)}")

#             # Clean up the test entry
#             cur.execute("DELETE FROM conversations WHERE id = 'test';")
#             conn.commit()
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         conn.rollback()
#     finally:
#         conn.close()

In [44]:
# RUN_TIMEZONE_CHECK

False

In [45]:
# if RUN_TIMEZONE_CHECK:
#     check_timezone()

# Prepare DB

In [63]:
get_db_connection(localhost=is_localhost)

<connection object at 0x1177e46d0; dsn: 'user=your_username password=xxx dbname=glimmerfox_db host=localhost', closed: 0>

In [64]:
init_db(localhost=is_localhost)

Table 'queries' created successfully.
Table 'feedback' created successfully.
Database initialization completed successfully.
Tables in the database: [('queries',), ('feedback',)]
Database connection closed.


In [83]:
clear_tables(localhost=is_localhost)

All entries in 'queries' and 'feedback' tables have been deleted.


# Generating historical data for 6 hours

In [79]:
# def generate_synthetic_data_old(start_time, end_time):
#     # clear_tables(localhost=is_localhost)
#     sample_queries = generate_sample_queries(20)

#     for query_id, question, answer_data in sample_queries:
#         save_query(query_id, question, answer_data)
#         save_feedback(
#             query_id=query_id, 
#             feedback=int(random.choice([True, False]))
#             )


In [73]:
def generate_synthetic_data(start_time, end_time, localhost=False):
    current_time = start_time
    queries_count = 0
    print(f"Starting historical data generation from {start_time} to {end_time}")
    while current_time < end_time:

        query_id, question, answer_data = generate_one_sample_query()
        
        save_query(query_id, question, answer_data, current_time, localhost=localhost)
        # print(
        #     f"Saved query: ID={query_id}, Time={current_time}"
        # )

        if random.random() < 0.7:
            feedback = 1 if random.random() < 0.8 else -1
            save_feedback(
                query_id=query_id, 
                feedback=int(random.choice([True, False])), 
                timestamp=current_time, 
                localhost=True)
            # print(
            #     f"Saved feedback for query {query_id}: {'Positive' if feedback > 0 else 'Negative'}"
            # )

        # current_time += timedelta(minutes=random.randint(1, 2))
        # current_time += timedelta(minutes=random.randint(1, 2))
        current_time += timedelta(seconds=random.randint(1, 20))
        queries_count += 1
        if queries_count % 10 == 0:
            print(f"Generated {queries_count} queries so far...")

    print(
        f"Historical data generation complete. Total queries: {queries_count}"
    )


In [82]:
print(f"Script started at {datetime.now(tz)}")
end_time = datetime.now(tz)
start_time = end_time - timedelta(hours=6)
print(f"Generating historical data from {start_time} to {end_time}")
generate_synthetic_data(start_time, end_time, localhost=True)
print("Historical data generation complete.")

Script started at 2024-09-14 16:02:13.232191+02:00
Generating historical data from 2024-09-14 10:02:13.232300+02:00 to 2024-09-14 16:02:13.232300+02:00
Starting historical data generation from 2024-09-14 10:02:13.232300+02:00 to 2024-09-14 16:02:13.232300+02:00
Generated 10 queries so far...
Generated 20 queries so far...
Generated 30 queries so far...
Generated 40 queries so far...
Generated 50 queries so far...
Generated 60 queries so far...
Generated 70 queries so far...
Generated 80 queries so far...
Generated 90 queries so far...
Generated 100 queries so far...
Generated 110 queries so far...
Generated 120 queries so far...
Generated 130 queries so far...
Generated 140 queries so far...
Generated 150 queries so far...
Generated 160 queries so far...
Generated 170 queries so far...
Generated 180 queries so far...
Generated 190 queries so far...
Generated 200 queries so far...
Generated 210 queries so far...
Generated 220 queries so far...
Generated 230 queries so far...
Generated 2

# Generating historical data

In [58]:
def generate_live_data(localhost=False):
    queries_count = 0
    print("Starting live data generation...")
    while True:
        current_time = datetime.now(tz)
        # current_time = None
        query_id, question, answer_data = generate_one_sample_query()
        save_query(query_id, question, answer_data, current_time, localhost=is_localhost)

        if random.random() < 0.7:
            feedback = 1 if random.random() < 0.8 else -1
            save_feedback(
                query_id=query_id, 
                feedback=int(random.choice([True, False])), 
                timestamp=current_time, 
                localhost=True)
        queries_count += 1
        if queries_count % 10 == 0:
            print(f"Generated {queries_count} live queries so far...")

        time.sleep(1)

In [70]:
print("Starting live data generation... Press Ctrl+C to stop.")
try:
    generate_live_data(localhost=is_localhost)
except KeyboardInterrupt:
    print(f"Generating historical data stopped at {datetime.now(tz)}.")
finally:
    print(f"Script ended at {datetime.now(tz)}")


Starting live data generation... Press Ctrl+C to stop.
Starting live data generation...
Generated 10 live queries so far...
Generated 20 live queries so far...
Generated 30 live queries so far...
Generated 40 live queries so far...
Generated 50 live queries so far...
Generated 60 live queries so far...
Generated 70 live queries so far...
Generating historical data stopped at 2024-09-14 15:55:23.416749+02:00.
Script ended at 2024-09-14 15:55:23.416850+02:00
